# Supply Chain Analysis for Order History in a Fashion Retail Store: Data Management with <img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*bWrto2YAmeGjEW9sZIkdcg.png" width = 300> and <img src="https://docs.delta.io/latest/_static/delta-lake-logo.png" width=200/>

# **Project Scenario**

As a **_Data Engineer_** for an online clothing brand offering a variety of **_fashion brands_**, I am developing a **_Supply Chain Dashboard_** to analyze **_Order History_**. This dashboard will support **_Purchasing Decisions_** and help maintain **_Sufficient Inventory Levels_** for the upcoming **_Holiday Season_**.

---

### **Data Ingestion Step**

I am **creating and ingesting fictional JSON file data** into the **Databricks file system**. This dataset will be used to support analysis for the **Supply Chain Dashboard** project. 

Using **Databricks**, I will load the data into a **Delta Table** to enable efficient processing and transformations, ultimately building a **scalable data pipeline** for actionable insights.

---

### **Project Steps**

1. **Create and Ingest Data** into a **Delta Table**.
2. Use **Databricks Notebooks** in **Python** and **SQL** to **Process and Transform** the data.
3. Develop a **Supply Chain Dashboard** for real-time insights.

---

### **Delta Lake Features**

Leverage **Delta Lake** functionalities, including:
- **Merge Operations** for efficient data updates.
- **Time Travel** to manage historical data views.

This approach will build a **Scalable Data Pipeline** that supports robust analytics and data integrity.

---



# Uploaded fictional JSON file data to the Databricks file system and Git Repo

### a. Upload ORDERS Json files in Databricks File System & Git Repo.

### b. Check loaded files

In [0]:
# Use Databricks Utilities (dbutils). Documentation : https://docs.databricks.com/dev-tools/databricks-utils.html#ls-command-dbutilsfsls 

dbutils.fs.ls("/Volumes/foraproject/default/filestore/SupplyChain/ORDERS_RAW/")





# Loading ORDERS_RAW Data Into Notepad and Creating Delta Table

### a. Read multiline json files using spark dataframe:

In [0]:
# Read multiple line json files using spark dataframeAPI

orders_raw_df = spark.read.option("multiline", "true").json("/Volumes/foraproject/default/filestore/SupplyChain/ORDERS_RAW/ORDERS_RAW_PART_*.json")


## Show the datafarme
orders_raw_df.show(n=5, truncate=False) 


In [0]:
#Validate loaded files Count Number of Rows in the DataFrame

orders_raw_df.count()

### ![b.](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) b. Create Delta Table ORDERS_RAW

Delta Lake is 100% compatible with Apache Spark&trade;, which makes it easy to get started with if you already use Spark for your big data workflows.
Delta Lake features APIs for **SQL**, **Python**, and **Scala**, so that you can use it in whatever language you feel most comfortable in.


   <img src="https://databricks.com/wp-content/uploads/2020/12/simplysaydelta.png" width=400/>

In [0]:
# First, Create Database SupplyChainDB if it doesn't exist
db = "SupplyChainDB"

spark.sql(f"CREATE DATABASE IF NOT EXISTS {db}")
spark.sql(f"USE {db}")

In [0]:
## Create DelaTable ORDERS_RAW in the metastore using DataFrame's schema and write data to it
## Documentation : https://docs.delta.io/latest/quick-start.html#create-a-table

orders_raw_df = spark.read.table("SupplyChainDB.ORDERS_RAW")
orders_raw_df.write.format("delta").mode("overwrite").saveAsTable("ORDERS_RAW")

### C. Show Created Delta Table:

In [0]:
%sql

SHOW TABLES

### d. Validate data loaded successfully to Delta Table ORDERS_RAW**:

In [0]:
%sql

SELECT count(*) FROM ORDERS_RAW

In [0]:
%sql

DESCRIBE TABLE ORDERS_RAW

# Transform data in delta table

<a href="https://www.databricks.com/glossary/medallion-architecture" target="_blank">Medallion Architecture</a>   </br><img src="https://databricks.com/wp-content/uploads/2020/09/delta-lake-medallion-model-scaled.jpg" width=900/>

### a. Read ORDERS_RAW delta table using spark Dataframe

In [0]:
#read Delta Table using spark dataframe

ORDERS_Gold_df= spark.read.table("SupplyChaindb.ORDERS_RAW")

ORDERS_Gold_df.show(n=5,truncate=False)
# Click on ORDERS_DF to See the Schema of the Table. 

### b. Update ORDER_DATE Column's Data Type

In [0]:
%python
from pyspark.sql import SparkSession

ORDERS_Gold_df = ORDERS_Gold_df.withColumn("ORDER_DATE", to_date(col("ORDER_DATE"), "yyyy-MM-dd"))

### c. Drop Rows with Null Values

In [0]:
# Count Nulls for each column
from pyspark.sql.functions import *

display(ORDERS_Gold_df.select([count(when(col(c).isNull(),c)).alias(c) for c in ORDERS_Gold_df.columns]))

In [0]:
#  Remove Nulls using dropna() method which removes all rows with Null Values 

ORDERS_Gold_df = ORDERS_Gold_df.dropna()

ORDERS_Gold_df.count()

In [0]:
# Count Nulls for each column
from pyspark.sql.functions import *

display(ORDERS_Gold_df.select([count(when(col(c).isNull(),c)).alias(c) for c in ORDERS_Gold_df.columns]))

### d. Add new Column TOTAL_ORDER

In [0]:
#Use withColumn function

ORDERS_Gold_df= ORDERS_Gold_df.withColumn("TOTAL_ORDER", col("QUANTITY") * col("UNIT_PRICE"))

# Display ORDERS_Gold_df to validate the creation of the New Column TOTAL_ORDER
display(ORDERS_Gold_df)

### e. Create Delta Table ORDERS_GOLD

In [0]:
spark.sql(f"USE SupplyChainDB")

## Create DeltaTable Orders_GOLD: 

ORDERS_Gold_df.write.mode("overwrite").format("delta").saveAsTable("ORDERS_GOLD")


## Validate that the table was created successfully
display(spark.sql(f"SHOW TABLES"))

In [0]:
%python
display(table("ORDERS_RAW"))

In [0]:
%python
display(table("ORDERS_Gold"))

# TASK 5 - Query Orders Delta table using SQL

### Get Orders_Gold dataset using SQL

In [0]:
%sql
-- Get top 30 rows Get Familiar with the Data

USE supplychainDB;

SELECT * FROM ORDERS_GOLD LIMIT 30;

### KPI-1: Quantity Sold by Country

In [0]:
%sql

SELECT ORDER_COUNTRY, SUM(QUANTITY) as TOTAL_DEMAND from supplychaindb.orders_gold WHERE ORDER_STATUS != 'CANCELLED' GROUP BY ORDER_COUNTRY

Databricks visualization. Run in Databricks to view.

### KPI-2: Sales by Division ($)

In [0]:
%sql

SELECT CATEGORY, SUM(TOTAL_ORDER) AS REVENUE FROM supplychaindb.orders_gold WHERE ORDER_STATUS != 'CANCELLED' GROUP BY CATEGORY ORDER BY REVENUE DESC;

Databricks visualization. Run in Databricks to view.

### KPI-3: Top-5 Popular Brands

In [0]:

%sql

SELECT BRAND, SUM(QUANTITY) AS TOTAL_SOLD_ITEMS from supplychaindb.orders_gold GROUP BY BRAND ORDER BY TOTAL_SOLD_ITEMS DESC LIMIT 10;

Databricks visualization. Run in Databricks to view.

### KPI-4: Demand by Month

In [0]:
%sql

SELECT date_trunc('MONTH',ORDER_DATE) as Month, SUM(QUANTITY) as Demand FROM supplychaindb.orders_gold WHERE ORDER_STATUS != "Canelled" GROUP BY Month ORDER BY Month DESC;

Databricks visualization. Run in Databricks to view.